In [207]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
from tensorflow.python.keras import models, layers, optimizers

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix
import re
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import RNN
from tensorflow.python.keras.layers import GRU
from tensorflow.keras.activations import tanh, sigmoid, relu, softmax
import warnings
from tensorflow.keras import models
warnings.simplefilter(action='ignore', category=FutureWarning)


import pickle
import sys
import warnings
import pickle

import os


def get_dataset():
    movie_reviews = pd.read_csv("IMDB Dataset.csv")
    train_texts = movie_reviews['review']
    sent = movie_reviews['sentiment']
    train_labels = list(map(lambda x: 1 if x == "positive" else 0, sent))

    print("Dataset size: %d" % len(train_labels))

    p = np.random.permutation(len(train_texts))
    train_texts = [train_texts[p[i]] for i in range(len(train_texts))]
    train_labels = [train_labels[p[i]] for i in range(len(train_labels))]

    # Normlaize

    TAG_RE = re.compile(r'<[^>]+>')

    def remove_tags(text):
        return TAG_RE.sub('', text)

    def preprocess_text(sen):
        # Removing html tags
        sentence = remove_tags(sen)

        # Remove punctuations and numbers
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)

        # Single character removal
        sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

        # Removing multiple spaces
        sentence = re.sub(r'\s+', ' ', sentence)

        return sentence

    print("Normalizing Dataset")

    NON_ALPHANUM = re.compile(r'[\W]')
    NON_ASCII = re.compile(r'[^a-z0-1\s]')

    test_ascii = []

    def normalize_texts(texts, labels):
        normalized_texts = []
        normalized_labels = []
        for i in range(len(texts)):
            text = texts[i]
            label = labels[i]

            lower = text.lower()

            lower = preprocess_text(lower)

            test_ascii.append(lower.split(" "))

            normalized_texts.append(lower)
            normalized_labels.append(label)
            continue

            no_punctuation = NON_ALPHANUM.sub(r' ', lower)
            no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
            normalized_texts.append(no_non_ascii)
        return normalized_texts, normalized_labels

    train_texts, train_labels = normalize_texts(train_texts, train_labels)

    train_labels = np.asarray(train_labels)

    # validation
    print("Test Set")

    MAX_FEATURES = 5000
    # print("MAX_FEATURES: %d" % MAX_FEATURES)
    tokenizer = Tokenizer(num_words=MAX_FEATURES)
    tokenizer.fit_on_texts(train_texts)
    num_words = len(tokenizer.word_index.items())
    MAX_FEATURES = num_words + 1

    test_texts = train_texts[:500]
    test_labels = train_labels[:500]
    train_texts = train_texts[500:]
    train_labels = train_labels[500:]
    test_ascii = test_ascii[:500]

    ##########################
    # ADD YOUR OWN TEST TEXT #
    ##########################

    my_test_texts = []
    my_test_texts.append("not extremely amazing good love like said")
    my_test_texts.append("very good movie i love movie very love ")
    my_test_texts.append("you are so capable strong amazing and good looking that even writing your own name is easy for you")
    my_test_texts.append("bad move i hate it made me feel gross and bad and shallow really boring")

    ##########################
    ##########################

    for k in range(len(my_test_texts)):
        test_texts[k] = my_test_texts[k]
        test_ascii[k] = my_test_texts[k].split(" ")

    train_texts = tokenizer.texts_to_sequences(train_texts)
    test_texts = tokenizer.texts_to_sequences(test_texts)

    # pad
    print("Padding")
    MAX_LENGTH = 100  # max(len(train_ex) for train_ex in train_texts)
    print("Max sentence length: %d" % MAX_LENGTH)
    train_texts = pad_sequences(train_texts, padding='post', maxlen=MAX_LENGTH)
    test_texts = pad_sequences(test_texts, padding='post', maxlen=MAX_LENGTH)

    # read embedding mat
    print("Read embedding matrix")
    embeddings_dictionary = dict()
    glove_file = open('glove.6B.100d.txt', encoding="utf8")
    word2ind=dict()
    i=0
    for line in glove_file:
        records = line.split()
        word = records[0]
        word2ind[word]=i
        i+=1
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary[word] = vector_dimensions
    glove_file.close()

    embedding_matrix = np.zeros((num_words + 1, 100))
    for word, index in tokenizer.word_index.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

    return train_texts, train_labels, test_texts, test_labels, test_ascii, embedding_matrix, MAX_LENGTH, MAX_FEATURES , embeddings_dictionary , word2ind , tokenizer


warnings.filterwarnings("ignore")

##########################
# Code for Ex. #2 in IDL #
##########################


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow as tf
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import sys

sys.setrecursionlimit(2500)

import os


train_texts, train_labels, test_texts, test_labels, test_ascii, embedding_matrix, MAX_LENGTH, MAX_FEATURES , emb_dict,word2ind,token=get_dataset()




Dataset size: 50000
Normalizing Dataset
Test Set
Padding
Max sentence length: 100
Read embedding matrix


In [164]:
#Debugging the given code to find out the sub scores bugs
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
a=token.word_index
b=list(token.word_index.items())[:25]
print(b)

print(test_ascii[0][:10])
print(test_texts[0][:10])
for ind,asci_word,test_text in zip(range(10),test_ascii[0][:10],test_texts[0][:10]):
    print("ascii word : {} , test word ind : {} , tokenizer ind : {}".format(asci_word,test_text,a[asci_word]))


[('the', 1), ('and', 2), ('of', 3), ('to', 4), ('is', 5), ('it', 6), ('in', 7), ('this', 8), ('that', 9), ('was', 10), ('as', 11), ('movie', 12), ('for', 13), ('with', 14), ('but', 15), ('film', 16), ('you', 17), ('on', 18), ('not', 19), ('he', 20), ('are', 21), ('his', 22), ('have', 23), ('one', 24), ('be', 25)]
['not', 'extremely', 'amazing', 'good', 'love', 'like', 'said']
[ 19 554 483  45 106  34 300   0   0   0]
ascii word : not , test word ind : 19 , tokenizer ind : 19
ascii word : extremely , test word ind : 554 , tokenizer ind : 554
ascii word : amazing , test word ind : 483 , tokenizer ind : 483
ascii word : good , test word ind : 45 , tokenizer ind : 45
ascii word : love , test word ind : 106 , tokenizer ind : 106
ascii word : like , test word ind : 34 , tokenizer ind : 34
ascii word : said , test word ind : 300 , tokenizer ind : 300


In [3]:
##########################
# Code for Ex. #2 in IDL #
##########################



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow as tf
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import sys
sys.setrecursionlimit(2500)

import os



#####################
# Execusion options #
#####################

# Getting activations from model

def get_act(net, input, name):
	sub_score = [layer for layer in net.layers if name in layer.name][0].output
	#functor = K.function([test_texts]+ [K.learning_phase()], sub_score)
	OutFunc = K.function([net.input], [sub_score])
	return OutFunc([test_texts])[0]
	
	
# RNN Cell Code

In [4]:

def RNN(dim, x):
    # Learnable weights in the cell
    Wh = layers.Dense(dim, use_bias=False)
    Wx = layers.Dense(dim)

    # unstacking the time axis
    x = tf.unstack(x, axis=1)

    H = []

    
    for i in range(len(x)):
        output_x=Wx(x[i])
        if i==0:
            h=Wh(Wx(x[i]))
        else:
            h=Wh(H[-1])
            
        H.append(relu(output_x+h))

    H = tf.stack(H, axis=1)

    return h, H

In [236]:
# GRU Cell Code

def GRU(dim, x):
    # Learnable weights in the cell
    Wzx = layers.Dense(dim)
    Uzh = layers.Dense(dim, use_bias=False)

    Wrx = layers.Dense(dim)
    Urh = layers.Dense(dim, use_bias=False)

    Whx = layers.Dense(dim)
    Uhh = layers.Dense(dim, use_bias=False)

    # unstacking the time axis
    x = tf.unstack(x, axis=1)

    H = []

    h = tf.zeros_like(Whx(x[0]))

    for i in range(len(x)):
        if i != 0 :
            h = H[-1]
        wzx = Wzx(x[i])
        wrx = Wrx(x[i])
        whx = Whx(x[i])
        z = sigmoid(wzx + Uzh(h))
        r = sigmoid(wrx + Urh(h))
        h_hat = tanh(whx + Uhh(h * r))
        h = (1 - z) * h + z * h_hat
        H.append(h)


    H = tf.stack(H, axis=1)

    return h, H

# (Spatially-)Restricted Attention Layer
# k - specifies the -k,+k neighbouring words


In [197]:
def restricted_attention(x,k):
    dim = x.shape[2]

    Wq = layers.Dense(dim)
    Wk = layers.Dense(dim)

    wk = Wk(x)

    paddings = tf.constant([[0, 0, ], [k, k], [0, 0]])
    pk = tf.pad(wk, paddings)
    pv = tf.pad(x, paddings)

    keys = []
    vals =   []
    for i in range(-k, k + 1):
        keys.append(tf.roll(pk, i, 1))
        vals.append(tf.roll(pv, i, 1))

    keys = tf.stack(keys, 2)
    keys = keys[:, k:-k, :, :]
    vals = tf.stack(vals, 2)
    vals = vals[:, k:-k, :, :]

    query = Wq(x)
    dot_product = tf.einsum('lij,likj->lik', query, keys) / np.sqrt(dim)
    attn_weights = layers.Softmax(name="atten_weights", axis=1)(dot_product)
    val_out = tf.einsum('lij,lijk->lik', attn_weights, vals)
    return x + val_out


In [201]:

TRAIN = True

RECR = True # recurrent netowrk (RNN/GRU) or a non-recurrent network

ATTN = False # use attention layer in global sum pooling or not
LSTM = True  # use LSTM or otherwise RNN


def build_model(dim_FC_RNN=64, dim_FC_GRU=64):
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedding_layer = layers.Embedding(MAX_FEATURES, 100, weights=[embedding_matrix], input_length=MAX_LENGTH,
                                       trainable=False)

    # embedding the words into 100 dim vectors

    x = embedding_layer(sequences)

    if not RECR:

        # non recurrent networks

        if ATTN:
            # attention layer
            x = restricted_attention(x, k=5)

        # word-wise FC layers -- MAKE SURE you have ,name= "sub_score" in the sub_scores step
        # E.g., sub_score = layers.Dense(2,name="sub_score")(x)

        x = tf.unstack(x, axis=1)
        net = tf.keras.Sequential()

        for i in range(10):
            net.add(layers.Dense(50, name="sub_score{}".format(i), activation="relu"))
            net.add(layers.Dropout(0.1))

        net.add(layers.Dense(32, activation="relu"))
        net.add(layers.Dense(2))

        weight_pred=[]
        for i in range(100):
            word_pred = net(x[i])
            weight_pred.append(word_pred)


        weights_tf = (tf.convert_to_tensor(weight_pred, dtype=tf.float32))

        preds=(weights_tf[:,:,0])
        weights=(weights_tf[:,:,1])

        weights_dist = softmax(weights,axis=0)
        weighted_pred =(preds * weights_dist)

        pre_pred = tf.math.reduce_sum(weighted_pred,axis=0)
        sig =sigmoid(pre_pred)
        predictions = tf.reshape(tf.convert_to_tensor(sig, dtype=tf.float32), shape=(-1, 1))

    else:
        # recurrent networks
        if LSTM:
            x, _ = GRU(dim_FC_GRU, x)
        else:
            x, _ = RNN(dim_FC_RNN, x)


        x = layers.Dense(32, activation='relu')(x)
        x = layers.Dense(1, activation='sigmoid')(x)

        predictions = x

    model = models.Model(inputs=sequences, outputs=predictions)

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model



In [237]:
def train(RNN_DIM=64,GRU_DIM=64): 
    model = build_model(RNN_DIM,GRU_DIM)
    if TRAIN:
        print("Training")
        History=model.fit(
            train_texts,
            train_labels,
            batch_size=128,
            epochs=15,validation_data=(test_texts,test_labels))
    return model , History

model,History=train(GRU_DIM=64)
preds = model.predict(test_texts)

In [237]:
# Comapre diffenrent inner dimensions for the model
# preds = model.predict(test_texts)
# acc=[]
# for i in np.arange(2,8)*10:
#     model=train(GRU_DIM=i)
#     preds = model.predict(test_texts)
#     print('Accuracy score: for inner dim {} {:0.4}'.format(i,accuracy_score(test_labels, 1 * (preds > 0.5))))
#     acc.append(accuracy_score(test_labels, 1 * (preds > 0.5)))

In [216]:
#Create a model that's cut at the 4th layer to get the predicitons for the subscores \ weights , it was needed to change to 39 if we're dealing with attention layers.

model2= models.Model(inputs=model.input, outputs=model.layers[4].output)


In [234]:
curr=3
sub_scores=model2.predict(test_texts[curr:curr+1])
current_sentence=pd.DataFrame(columns=["score"])
for i in range(1):

    print("-" * 20)

    if not RECR: 
        num = min((len(test_ascii[curr]), 100))
        for k in range(num):
            current_sentence.loc[test_ascii[curr][k]]=sub_scores[k,0,0]
            print("Scores for the word:  '{}'  with the following score {}".format(test_ascii[curr][k],sub_scores[k,0,0]))
        print("\n")
    else:
        print(test_ascii[i])
        print(preds[i])

    if preds[curr] > 0.5:
        print("Positive")
    else:
        print("Negative")
    print("-" * 20)


--------------------
Scores for the word:  'bad'  with the following score -13.457328796386719
Scores for the word:  'move'  with the following score 2.854335308074951
Scores for the word:  'i'  with the following score 2.8893215656280518
Scores for the word:  'hate'  with the following score 1.9233105182647705
Scores for the word:  'it'  with the following score 2.416504144668579
Scores for the word:  'made'  with the following score -6.323531150817871
Scores for the word:  'me'  with the following score 4.552388668060303
Scores for the word:  'feel'  with the following score 7.098709583282471
Scores for the word:  'gross'  with the following score 2.6295955181121826
Scores for the word:  'and'  with the following score 5.257828712463379
Scores for the word:  'bad'  with the following score -13.457328796386719
Scores for the word:  'and'  with the following score 5.257828712463379
Scores for the word:  'shallow'  with the following score -16.907142639160156
Scores for the word:  'real

,bad,move,i,hate,it,made,me,feel,gross,and,shallow,really,boring
score,-13.457329,2.854335,2.889322,1.923311,2.416504,-6.323531,4.552389,7.09871,2.629596,5.257829,-16.907143,2.05943,-21.266457


In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7,4)

In [14]:
model.History["binary_accuracy"]

AttributeError: 'Functional' object has no attribute 'History'

In [ ]:
sns.set()
sns.lineplot(np.arange(EPOCH_NUM),History.history["binary_accuracy"],label="train acc")
sns.lineplot(np.arange(EPOCH_NUM),History.history["val_binary_accuracy"],label="test acc")
plt.title("Q1.3 training process accuracy")
plt.show()

In [ ]:
tf.config.list_physical_devices('GPU')